In [55]:
import pandas as pd
from xgboost import XGBClassifier as XGBC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix as cm, roc_auc_score as auc

In [2]:
data = pd.read_csv("Kaggle_Sirio_Libanes_ICU_Prediction.csv")
print(data.shape)
print(data.info(verbose=True,null_counts=True))

(1925, 231)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1925 entries, 0 to 1924
Data columns (total 231 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   PATIENT_VISIT_IDENTIFIER          1925 non-null   int64  
 1   AGE_ABOVE65                       1925 non-null   int64  
 2   AGE_PERCENTIL                     1925 non-null   object 
 3   GENDER                            1925 non-null   int64  
 4   DISEASE GROUPING 1                1920 non-null   float64
 5   DISEASE GROUPING 2                1920 non-null   float64
 6   DISEASE GROUPING 3                1920 non-null   float64
 7   DISEASE GROUPING 4                1920 non-null   float64
 8   DISEASE GROUPING 5                1920 non-null   float64
 9   DISEASE GROUPING 6                1920 non-null   float64
 10  HTN                               1920 non-null   float64
 11  IMMUNOCOMPROMISED                 1920 non-null   float6

In [9]:
data['LACTATE_MEAN'].describe()

count    821.000000
mean       0.267131
std        0.923557
min       -1.000000
25%       -0.894695
50%        1.000000
75%        1.000000
max        1.000000
Name: LACTATE_MEAN, dtype: float64

In [6]:

data_y = data.loc[:,'ICU']


respiratory rate, lactate, blood pressure(diastolic and systolic), neutrophils, oxygen saturation level, immature grans(Abs),
hemoglobin, procalcitonin, erythrocyte sedimentation rates, brain natriuretic peptide, ferritin, D-Dimer, platelets
hematocrit, base excess venous and arterial, albumin, urea

In [15]:
x_column = ['RESPIRATORY_RATE_MEAN'
            ,'LACTATE_MEAN'
            ,'BLOODPRESSURE_DIASTOLIC_MEAN'
            ,'BLOODPRESSURE_SISTOLIC_MEAN'
           ,'NEUTROPHILES_MEAN'
            ,'OXYGEN_SATURATION_MEAN'
            ,'HEMOGLOBIN_MEAN'
           ,'UREA_MEAN'
           ,'DIMER_MEAN'
           ,'PLATELETS_MEAN'
           ,'HEMATOCRITE_MEAN']

In [16]:
data_x = data.loc[:,x_column]

In [45]:
Xtrain,Xtest,Ytrain,Ytest = train_test_split(data_x,data_y,test_size=0.3,random_state=111)


In [46]:
print(Xtrain.shape)
print(Xtest.shape)

(1347, 11)
(578, 11)


In [48]:
clf = XGBC().fit(Xtrain,Ytrain)




In [59]:
ypred = clf.predict(Xtest)
accuracy = accuracy_score(Ytest,ypred)
precision = precision_score(Ytest,ypred)
recall = recall_score(Ytest,ypred)
f1 = f1_score(Ytest,ypred)
print("accuracy",accuracy)
print("precision",precision)
print("recall",recall)
print("f1",f1)

accuracy 0.870242214532872
precision 0.7698412698412699
recall 0.6783216783216783
f1 0.721189591078067


In [53]:
recall(Ytest,ypred)
auc(Ytest,clf.predict_proba(Xtest)[:,1])

0.8694558315247969

In [49]:
#负/正样本比例
clf_ = XGBC(scale_pos_weight=10).fit(Xtrain,Ytrain)
ypred_ = clf_.predict(Xtest)
clf_.score(Xtest,Ytest)
cm(Ytest,ypred_,labels=[1,0])
recall(Ytest,ypred_)
auc(Ytest,clf_.predict_proba(Xtest)[:,1])

0.8540390643838919

In [ ]:
#随着样本权重逐渐增加，模型的recall,auc和准确率如何变化？
for i in [1,5,10,20,30]:
clf_ = XGBC(scale_pos_weight=i).fit(Xtrain,Ytrain)